## Tensorflow

In [311]:
import pandas as pd
import numpy as np 
import nltk
import string
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nltk
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc,accuracy_score,f1_score,precision_score,recall_score
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC,Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.svm import LinearSVC
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import precision_score,recall_score,auc



In [312]:
import warnings
warnings.filterwarnings("ignore")

In [313]:
train_data = pd.read_csv("/Users/sarath/Documents/Final Project Data/train_values.csv")
train_labels = pd.read_csv("/Users/sarath/Documents/Final Project Data/train_labels.csv")
test_labels = pd.read_csv("/Users/sarath/Documents/Final Project Data/test_values.csv")

In [314]:
building_damage1 = train_data.merge(train_labels, how = 'inner', on = 'building_id')

In [315]:
from functools import reduce

In [316]:
building_damage1 = building_damage1.drop(columns ="has_secondary_use")

In [317]:
building_damage1 = building_damage1.drop(columns ="has_secondary_use_agriculture")

In [318]:
building_damage = building_damage1[building_damage1['age'] <= 250]

In [319]:
 #utility function to preprocess data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer,LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
def preprocess_num_data(X_train,num_data,X_test):
    std=Normalizer()
    std.fit(train_data)
    transformed_input=std.transform(building_damage)
    transformed_test=std.transform(test_labels)
    return transformed_input,transformed_test
def preprocess_cat_data(building_damage,test_labels):
    std=LabelEncoder()
    for col in train_data.columns:
        col_list_train=list(map(str,building_damage[col].values))
        #print(col_list_train)
        col_list_test=list(map(str,test_lables[col].values))
        std.fit(col_list_train)
        col_list_train=std.transform(col_list_train)
        col_list_test=std.transform(col_list_test)
        building_damage[col]=col_list_train
        test_data[col]=col_list_test
    return building_damage,test_data

In [320]:
num_data=['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq','area_percentage', 'age','height_percentage','count_families']
cat_data=['land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position','legal_ownership_status',
       'plan_configuration'] 
bin_data=['has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
         'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other']

In [321]:
# filtering numerical data
num_train=building_damage[num_data]
num_test=test_labels[num_data]
# filtering categorical data
cat_train=building_damage[cat_data]
cat_test=building_damage[cat_data]
# filtering binary data
bin_train=building_damage[bin_data]
bin_test=test_labels[bin_data]


In [322]:
cat_feats = ['land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position','legal_ownership_status',
       'plan_configuration']

In [323]:
building_damage['damage_grade'] = building_damage['damage_grade'].astype('category')

In [324]:
train_final = pd.get_dummies(building_damage,columns=cat_feats,drop_first=True)
test_final = pd.get_dummies(test_labels,columns=cat_feats,drop_first=True)

In [325]:
from sklearn.model_selection import train_test_split

In [326]:
y_train=train_final.damage_grade
train=train_final.drop('damage_grade',axis=1)

In [327]:
X = train_final.drop('damage_grade',axis=1)
y = train_final['damage_grade']

train_final.shape

(259211, 60)

In [328]:
scaler = StandardScaler()

TypeError: object of type 'CategoricalDtype' has no len()

In [329]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=5)

In [330]:
X_train

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,...,legal_ownership_status_w,plan_configuration_c,plan_configuration_d,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u
259171,102946,6,187,4685,2,20,7,7,0,1,...,0,0,1,0,0,0,0,0,0,0
114533,32468,10,735,10221,3,5,17,10,0,0,...,1,0,1,0,0,0,0,0,0,0
12011,730104,13,305,2545,1,5,5,4,0,1,...,0,0,1,0,0,0,0,0,0,0
114920,194343,10,1278,11384,2,30,6,4,0,1,...,0,0,1,0,0,0,0,0,0,0
83520,344994,20,788,1049,1,15,8,2,0,1,...,0,0,1,0,0,0,0,0,0,0
212585,5541,13,1262,12264,3,10,8,5,0,1,...,0,0,1,0,0,0,0,0,0,0
143439,315872,10,127,10118,2,20,4,5,1,1,...,0,0,1,0,0,0,0,0,0,0
43045,262073,8,1373,9157,3,20,7,9,0,1,...,0,0,1,0,0,0,0,0,0,0
252047,995381,11,765,8332,2,20,11,7,1,1,...,0,0,1,0,0,0,0,0,0,0
232235,413504,7,10,4605,2,15,4,6,0,1,...,0,0,1,0,0,0,0,0,0,0


In [331]:
y_train.value_counts()

2    110477
3     65238
1     18693
Name: damage_grade, dtype: int64

In [367]:


from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

scaled_features_train = sc.fit_transform(X_train)
scaled_features_test = sc.transform(X_test)



In [376]:
X_test

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,...,legal_ownership_status_w,plan_configuration_c,plan_configuration_d,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u
164260,417135,26,264,453,1,5,4,4,0,0,...,0,0,1,0,0,0,0,0,0,0
57902,1025894,7,1328,11760,2,20,9,6,0,1,...,0,0,1,0,0,0,0,0,0,0
248103,527391,26,1001,12258,1,10,5,2,0,0,...,0,0,1,0,0,0,0,0,0,0
85689,35182,4,148,10297,2,5,7,5,0,1,...,0,0,1,0,0,0,0,0,0,0
239767,80679,21,1051,10239,3,20,5,7,0,1,...,0,0,1,0,0,0,0,0,0,0
220241,424429,8,979,9416,2,5,4,5,0,1,...,1,0,1,0,0,0,0,0,0,0
47707,90578,27,533,5435,2,0,13,6,1,0,...,0,0,1,0,0,0,0,0,0,0
5042,1035252,26,1001,171,1,0,11,2,0,0,...,0,0,1,0,0,0,0,0,0,0
45841,427003,17,1357,3408,1,80,4,2,0,1,...,0,0,1,0,0,0,0,0,0,0
131530,325009,8,145,3389,1,0,4,2,0,0,...,0,0,1,0,0,0,0,0,0,0


In [377]:
X_train = pd.DataFrame(scaled_features_train, index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(scaled_features_test, index=X_test.index, columns=X_test.columns)


In [379]:
X_test

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,...,legal_ownership_status_w,plan_configuration_c,plan_configuration_d,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u
164260,-0.357837,1.507894,-1.056726,-1.592923,-1.551547,-0.832203,-0.915641,-0.749225,-0.310691,-1.788946,...,-0.101618,-0.03501,0.204666,-0.009072,-0.013226,-0.011112,-0.024749,-0.148886,-0.03617,-0.119276
57902,1.640298,-0.858620,1.520034,1.508410,-0.177802,-0.068263,0.223250,0.296117,-0.310691,0.558988,...,-0.101618,-0.03501,0.204666,-0.009072,-0.013226,-0.011112,-0.024749,-0.148886,-0.03617,-0.119276
248103,0.004058,1.507894,0.728116,1.645004,-1.551547,-0.577556,-0.687863,-1.794567,-0.310691,-1.788946,...,-0.101618,-0.03501,0.204666,-0.009072,-0.013226,-0.011112,-0.024749,-0.148886,-0.03617,-0.119276
85689,-1.611524,-1.232281,-1.337651,1.107132,-0.177802,-0.832203,-0.232307,-0.226554,-0.310691,0.558988,...,-0.101618,-0.03501,0.204666,-0.009072,-0.013226,-0.011112,-0.024749,-0.148886,-0.03617,-0.119276
239767,-1.462189,0.885127,0.849204,1.091223,1.195943,-0.068263,-0.687863,0.818788,-0.310691,0.558988,...,-0.101618,-0.03501,0.204666,-0.009072,-0.013226,-0.011112,-0.024749,-0.148886,-0.03617,-0.119276
220241,-0.333895,-0.734067,0.674837,0.865487,-0.177802,-0.832203,-0.915641,-0.226554,-0.310691,0.558988,...,9.840730,-0.03501,0.204666,-0.009072,-0.013226,-0.011112,-0.024749,-0.148886,-0.03617,-0.119276
47707,-1.429697,1.632447,-0.405271,-0.226439,-0.177802,-1.086850,1.134362,0.296117,3.218631,-1.788946,...,-0.101618,-0.03501,0.204666,-0.009072,-0.013226,-0.011112,-0.024749,-0.148886,-0.03617,-0.119276
5042,1.671014,1.507894,0.728116,-1.670271,-1.551547,-1.086850,0.678806,-1.794567,-0.310691,-1.788946,...,-0.101618,-0.03501,0.204666,-0.009072,-0.013226,-0.011112,-0.024749,-0.148886,-0.03617,-0.119276
45841,-0.325447,0.386913,1.590265,-0.782413,-1.551547,2.987496,-0.915641,-1.794567,-0.310691,0.558988,...,-0.101618,-0.03501,0.204666,-0.009072,-0.013226,-0.011112,-0.024749,-0.148886,-0.03617,-0.119276
131530,-0.660223,-0.734067,-1.344916,-0.787624,-1.551547,-1.086850,-0.915641,-1.794567,-0.310691,-1.788946,...,-0.101618,-0.03501,0.204666,-0.009072,-0.013226,-0.011112,-0.024749,-0.148886,-0.03617,-0.119276


### Tensorflow

In [380]:
import tensorflow as tf

In [381]:
X.columns

Index(['building_id', 'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'count_families', 'has_secondary_use_hotel', 'has_secondary_use_rental',
       'has_secondary_use_institution', 'has_secondary_use_school',
       'has_secondary_use_industry', 'has_secondary_use_health_post',
       'has_secondary_use_gov_office', 'has_secondary_use_use_police',
       'has_secondary_use_other', 'land_surface_condition_o',
       'land_surface_condition_t', 'foundation_type_i', 'f

In [382]:
building_id = tf.feature_column.numeric_column("building_id")
geo_level_1_id = tf.feature_column.numeric_column('geo_level_1_id')
geo_level_2_id = tf.feature_column.numeric_column('geo_level_2_id')
geo_level_3_id =tf.feature_column.numeric_column('geo_level_3_id')
count_floors_pre_eq = tf.feature_column.numeric_column("count_floors_pre_eq")
age = tf.feature_column.numeric_column('age')
area_percentage = tf.feature_column.numeric_column('area_percentage')
height_percentage =tf.feature_column.numeric_column('height_percentage')
has_superstructure_adobe_mud = tf.feature_column.numeric_column("has_superstructure_adobe_mud")
has_superstructure_mud_mortar_stone = tf.feature_column.numeric_column('has_superstructure_mud_mortar_stone')
has_superstructure_stone_flag = tf.feature_column.numeric_column('has_superstructure_stone_flag')
has_superstructure_cement_mortar_stone =tf.feature_column.numeric_column('has_superstructure_cement_mortar_stone')
has_superstructure_mud_mortar_brick = tf.feature_column.numeric_column('has_superstructure_mud_mortar_brick')
has_superstructure_timber = tf.feature_column.numeric_column('has_superstructure_timber')
has_superstructure_bamboo = tf.feature_column.numeric_column('has_superstructure_bamboo')
has_superstructure_rc_non_engineered =tf.feature_column.numeric_column('has_superstructure_rc_non_engineered')
has_superstructure_other = tf.feature_column.numeric_column("has_superstructure_other")
count_families = tf.feature_column.numeric_column('count_families')
has_secondary_use_hotel = tf.feature_column.numeric_column('has_secondary_use_hotel')
has_secondary_use_rental =tf.feature_column.numeric_column('has_secondary_use_rental')
has_secondary_use_institution = tf.feature_column.numeric_column("has_secondary_use_institution")
has_secondary_use_school = tf.feature_column.numeric_column('has_secondary_use_school')
has_secondary_use_industry = tf.feature_column.numeric_column('has_secondary_use_industry')
has_secondary_use_health_post =tf.feature_column.numeric_column('has_secondary_use_health_post')
has_secondary_use_gov_office = tf.feature_column.numeric_column("has_secondary_use_gov_office")
has_secondary_use_use_police = tf.feature_column.numeric_column('has_secondary_use_use_police')
has_secondary_use_other = tf.feature_column.numeric_column('has_secondary_use_other')
land_surface_condition_o =tf.feature_column.numeric_column('land_surface_condition_o')
land_surface_condition_t = tf.feature_column.numeric_column("land_surface_condition_t")
foundation_type_i = tf.feature_column.numeric_column('foundation_type_i')
foundation_type_r = tf.feature_column.numeric_column('foundation_type_r')
foundation_type_u =tf.feature_column.numeric_column('foundation_type_u')
foundation_type_w = tf.feature_column.numeric_column('foundation_type_w')
roof_type_q = tf.feature_column.numeric_column('roof_type_q')
roof_type_x = tf.feature_column.numeric_column('roof_type_x')
ground_floor_type_m =tf.feature_column.numeric_column('ground_floor_type_m')
ground_floor_type_v = tf.feature_column.numeric_column("ground_floor_type_v")
ground_floor_type_x = tf.feature_column.numeric_column('ground_floor_type_x')
ground_floor_type_z = tf.feature_column.numeric_column('ground_floor_type_z')
other_floor_type_q =tf.feature_column.numeric_column('other_floor_type_q')
other_floor_type_s =tf.feature_column.numeric_column('other_floor_type_s')
other_floor_type_x =tf.feature_column.numeric_column('other_floor_type_x')
position_o =tf.feature_column.numeric_column('position_o')
position_s =tf.feature_column.numeric_column('position_s')
position_t =tf.feature_column.numeric_column('position_t')
legal_ownership_status_r =tf.feature_column.numeric_column('legal_ownership_status_r')
legal_ownership_status_v =tf.feature_column.numeric_column('legal_ownership_status_v')
legal_ownership_status_w =tf.feature_column.numeric_column('legal_ownership_status_w')
plan_configuration_c =tf.feature_column.numeric_column('plan_configuration_c')
plan_configuration_d =tf.feature_column.numeric_column('plan_configuration_d')
plan_configuration_f =tf.feature_column.numeric_column('plan_configuration_f')
plan_configuration_m =tf.feature_column.numeric_column('plan_configuration_m')
plan_configuration_n =tf.feature_column.numeric_column('plan_configuration_n')
plan_configuration_o =tf.feature_column.numeric_column('plan_configuration_o')
plan_configuration_q =tf.feature_column.numeric_column('plan_configuration_q')
plan_configuration_s =tf.feature_column.numeric_column('plan_configuration_s')
plan_configuration_u =tf.feature_column.numeric_column('plan_configuration_u')




In [383]:
#y_train = tf.cast(y_train, tf.int32)


#y_train = pd.Series(y_train)

#y_train = y_train.astype("category")

y_train

259171    2
114533    2
12011     2
114920    2
83520     2
212585    2
143439    3
43045     3
252047    2
232235    3
11432     2
248709    2
104021    2
147480    2
12427     3
152609    1
64147     2
27625     3
14210     3
179279    3
162178    3
89726     1
72549     2
102464    3
62398     2
216155    3
28506     2
146329    2
115099    2
205168    3
         ..
166717    2
234112    3
126687    3
197328    2
206086    2
108117    3
63846     3
159644    1
193497    3
227939    2
233792    3
164594    1
160793    2
205374    2
20270     2
200373    3
52098     2
167174    3
34923     2
213828    2
165450    3
33971     2
59781     2
137324    3
122614    2
233653    2
125260    2
20559     3
18722     2
35864     3
Name: damage_grade, Length: 194408, dtype: int64

In [384]:
feat_cols = [building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,
            has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,
            has_superstructure_mud_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,
            has_superstructure_other,count_families,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,
            has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,
            has_secondary_use_other,land_surface_condition_o,land_surface_condition_t,foundation_type_i,foundation_type_r,foundation_type_u,
            foundation_type_w,roof_type_q,roof_type_x,ground_floor_type_m,ground_floor_type_v,ground_floor_type_x,ground_floor_type_z,other_floor_type_q,
            other_floor_type_s,other_floor_type_x,position_o,position_s,position_t,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w,
            plan_configuration_c,plan_configuration_d,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,
            plan_configuration_s,plan_configuration_u
            ]

In [385]:
classifier = tf.estimator.DNNClassifier(hidden_units=[10, 20, 10], n_classes=4,feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/wy/03xmxjqn6j3_dl74gsfyqg7c0000gn/T/tmpvig27uo0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a49bcdef0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [386]:
#y_train = list(y_train)
y_train = y_train.astype('int')

y_train

259171    2
114533    2
12011     2
114920    2
83520     2
212585    2
143439    3
43045     3
252047    2
232235    3
11432     2
248709    2
104021    2
147480    2
12427     3
152609    1
64147     2
27625     3
14210     3
179279    3
162178    3
89726     1
72549     2
102464    3
62398     2
216155    3
28506     2
146329    2
115099    2
205168    3
         ..
166717    2
234112    3
126687    3
197328    2
206086    2
108117    3
63846     3
159644    1
193497    3
227939    2
233792    3
164594    1
160793    2
205374    2
20270     2
200373    3
52098     2
167174    3
34923     2
213828    2
165450    3
33971     2
59781     2
137324    3
122614    2
233653    2
125260    2
20559     3
18722     2
35864     3
Name: damage_grade, Length: 194408, dtype: int64

In [387]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=20,shuffle=True)

In [388]:
classifier.train(input_fn=input_func,steps=500)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/wy/03xmxjqn6j3_dl74gsfyqg7c0000gn/T/tmpvig27uo0/model.ckpt.
INFO:tensorflow:loss = 33.415043, step = 1
INFO:tensorflow:global_step/sec: 97.7232
INFO:tensorflow:loss = 19.114325, step = 101 (1.024 sec)
INFO:tensorflow:global_step/sec: 184.504
INFO:tensorflow:loss = 15.545588, step = 201 (0.543 sec)
INFO:tensorflow:global_step/sec: 170.109
INFO:tensorflow:loss = 15.598955, step = 301 (0.587 sec)
INFO:tensorflow:global_step/sec: 151.918
INFO:tensorflow:loss = 17.52113, step = 401 (0.659 sec)
INFO:tensorflow:Saving checkpoints for 500 into /var/folders/wy/03xmxjqn6j3_dl74gsfyqg7c0000gn/T/tmpvig27uo0/model.ckpt.
INFO:tensorflow:Loss for final step: 15.832388.


### Model Evaluation

In [389]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [390]:
pred_fn

<function tensorflow_estimator.python.estimator.inputs.pandas_io.pandas_input_fn.<locals>.input_fn()>

In [391]:
note_predictions = list(classifier.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/wy/03xmxjqn6j3_dl74gsfyqg7c0000gn/T/tmpvig27uo0/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [392]:
note_predictions[0]

{'logits': array([-3.8568375 ,  0.49845418,  1.3135304 ,  0.40167844], dtype=float32),
 'probabilities': array([0.0030715 , 0.23923755, 0.54052085, 0.21717016], dtype=float32),
 'class_ids': array([2]),
 'classes': array([b'2'], dtype=object)}

In [393]:
final_preds  = []
for pred in note_predictions:
    final_preds.append(pred['class_ids'][0])

In [394]:
pd.Series(final_preds).value_counts()

2    54073
3     9303
1     1427
dtype: int64

In [395]:
from sklearn.metrics import classification_report,confusion_matrix

[2,
 3,
 2,
 2,
 3,
 3,
 2,
 2,
 3,
 2,
 3,
 2,
 3,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 3,
 2,
 3,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 3,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 3,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 2,
 3,
 2,
 2,
 2,
 2,
 1,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 3,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 2,
 2,
 2,
 3,
 3,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,


In [397]:
print(confusion_matrix(y_test,final_preds))

[[  811  5250   191]
 [  583 31906  4471]
 [   33 16917  4641]]


In [398]:
print('Accuracy: ',accuracy_score(y_test,final_preds))
print('Kappa:    ', cohen_kappa_score(final_preds,y_test))

Accuracy:  0.5764856565282471


NameError: name 'cohen_kappa_score' is not defined